In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.metrics import classification_report

#### Loading the Data

In [2]:
df = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [3]:
print("Shape : ", df.shape)
df.head()

Shape :  (5000, 14)


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


#### Data Cleaning and Feature Engineering

In [4]:
# Deleting Columns which are not necessary
df.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [5]:
print("Duplicate : ", df.duplicated().sum())
print("Null : ", df.isnull().sum().sum())

Duplicate :  13
Null :  0


In [6]:
print(df["Experience"].unique())
df["Experience"] = abs(df["Experience"])

[ 1 19 15  9  8 13 27 24 10 39  5 23 32 41 30 14 18 21 28 31 11 16 20 35
  6 25  7 12 26 37 17  2 36 29  3 22 -1 34  0 38 40 33  4 -2 42 -3 43]


In [7]:
df.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [8]:
df = df[['Age', 'Experience', 'Income', 'Family', 'CCAvg','Education', 'Mortgage', 'Securities Account','CD Account', 'Online', 'CreditCard', 'Personal Loan']]

In [9]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

### Train Test Split

In [10]:
x,y = X,Y

In [11]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=69)

In [12]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
x_train.shape, y_train.shape

((3750, 11), (3750,))

# PyTorch 

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [15]:
BATCH_SIZE = 1

In [16]:
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)

In [17]:
train_x.shape, train_y.shape

(torch.Size([3750, 11]), torch.Size([3750]))

In [18]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

## Building Model

In [19]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,16)
        self.layer2 = torch.nn.Linear(16,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [20]:
model = Model()
# model.to(device)
print(model)

Model(
  (layer1): Linear(in_features=11, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


# Weight Optimization using Ant Colony Optimization

In [21]:
torch.manual_seed(6699)
torch.set_grad_enabled(False)

In [22]:
dummy = Model()
dummy_param = np.concatenate([i.numpy().flatten() for i in dummy.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]
dim = len(dummy_param)

print("Dim : ", dim)
print("Layers Shape : ", shape)
print("Layers Size : ", size)

dummy = None
dummy_param = None

Dim :  209
Layers Shape :  [(16, 11), (16,), (1, 16), (1,)]
Layers Size :  [176, 16, 16, 1]


In [23]:
# Helper Functions

def model_to_vector(model):
    vector = np.concatenate([i.numpy().flatten() for i in model.parameters()])
    return vector
    
def vector_to_model(vector):
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = vector[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    
    dummy_model = Model()
    for idx,wei in enumerate(dummy_model.parameters()):
        wei.data = (torch.tensor(param[idx])).to(torch.float32)
        
    return dummy_model    

def calc_accuracy(model):
    y_pred = model(train_x)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
    
    return acc

In [24]:
# Tunable Parameters

ants = 10
loops = 100
evaporation_rate = 0.2
influence_factor = 0.4

In [25]:
pheromones = np.ones(dim)
max_accuracy = 0
fittest_vector = None

for loop in range(loops):
    # Generate Solution
    paths = np.array([Model() for i in range(ants)])
    accuracy = []
    
    for ant in range(ants):
        # Flatten the weights and biases
        vector = model_to_vector(paths[ant])
        
        # Multiply with pheromones 
        vector = vector * pheromones
        
        # Calculate Accuracy and Append to the list
        model = vector_to_model(vector)
        acc = calc_accuracy(model)
        accuracy.append(acc)
        
        # Update the updated path
        paths[ant] = model
        
        # Reset
        model = None
        acc = None
        
    # Select fittest path and accuracy
    paths = paths[np.argsort(accuracy)]
    
    if accuracy[np.argmax(accuracy)] > max_accuracy:
        max_accuracy = accuracy[np.argmax(accuracy)]
        fittest_vector = model_to_vector(paths[-1])
    
    # Update pheromones
    delta = 0
    for ant in range(ants):
        # Flatten the weights and biases
        vector = model_to_vector(paths[ant])
        
        # Calculate delta
        delta += (vector - fittest_vector)*influence_factor
        
    pheromones = (1-pheromones)*evaporation_rate + delta
    
    if loop%10 == 0:
        print("Iters {} :".format(loop), calc_accuracy(paths[-1]))

Iters 0 : 0.9016
Iters 10 : 0.9026666666666666
Iters 20 : 0.9026666666666666
Iters 30 : 0.9026666666666666
Iters 40 : 0.9029333333333334
Iters 50 : 0.9026666666666666
Iters 60 : 0.9026666666666666
Iters 70 : 0.9130666666666667
Iters 80 : 0.9026666666666666
Iters 90 : 0.9021333333333333


In [26]:
print("Maximum Accuracy : ", max_accuracy)
best_model = vector_to_model(fittest_vector)

Maximum Accuracy :  0.9232


## Testing 

In [27]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)

In [28]:
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

### Classification Report

In [29]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1143
           1       0.56      0.60      0.58       107

    accuracy                           0.92      1250
   macro avg       0.76      0.78      0.77      1250
weighted avg       0.93      0.92      0.93      1250

